In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from glob import glob
import pandas as pd
import pickle
import random
from PIL import Image
from glob import glob
import re
#from adjustText import adjust_text
import seaborn as sns
import statannot
import argparse
#import torch
import wandb
import sys

In [2]:
def combine_into_one_df(pattern, get_label_num = None):
    listo = glob(pattern)

    dice_sens = []
    dice_spec = []
    overall_dice = []
    labels = []

    for x in listo:
        this_pkl = pd.read_pickle(x)
        if get_label_num is None:
            label_num = int(x.split('/')[-2].split('label')[1])
        else:
            label_num = get_label_num(x)

        labels.append(label_num)
        dice_sens.append(this_pkl['dice_sensitivity'].item())
        dice_spec.append(this_pkl['dice_specificity'].item())
        overall_dice.append(this_pkl['overall_dice'].item())

    total_df = pd.DataFrame({'dice_sensitivity': dice_sens,
                            'dice_specificity': dice_spec,
                            'overall_dice': overall_dice,
                            'label_numbers': labels})
    total_df = total_df.sort_values(by = 'label_numbers')
    return total_df
        


In [3]:
#combine_into_one_pkl('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_eval_results_val_7-13-24/label*/eval_singletask_unet_val_label*.pkl')
df_high = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_eval_results_lr1e-4_val_7-30-24/label*/eval_singletask_unet_val_label*.pkl')
df_low = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_eval_results_lr1e-5_val_7-30-24/label*/eval_singletask_unet_val_label*.pkl')

df_high.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_eval_results_lr1e-4_val_7-30-24/combined_val.pkl')
df_low.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_eval_results_lr1e-5_val_7-30-24/combined_val.pkl')


In [ ]:
df_combined_medsam = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/medsam_retrain_9-12-24/val/*/0.001/eval_singletask_unprompted_val_label*.pkl', get_label_num = lambda x: int(x.split('/')[-3]))
df_combined_medsam = df_combined_medsam.reset_index(drop=True)
df_combined_medsam
#df_combined_medsam.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/medsam_retrain_9-12-24/val/combined_val.pkl')


In [8]:
df_combined_unet = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_retrain_all_labels_9-9-24/val/*/0.0005/eval_singletask_unet_val_label*.pkl', get_label_num = lambda x: int(x.split('/')[-3]))
#df_combined_unet
#df_combined_unet.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_retrain_all_labels_9-9-24/val/combined_val.pkl')

,dice_sensitivity,dice_specificity,overall_dice,label_numbers
48,0.942036,0.997877,0.974439,1
37,0.900616,0.995360,0.984680,2
21,0.749551,0.977894,0.965479,3
47,0.735495,0.993772,0.950026,4
36,0.893992,0.994133,0.970275,5
...,...,...,...,...
94,0.774828,0.989722,0.941844,98
60,0.779624,0.980553,0.938884,99
61,0.776131,0.985879,0.951246,100
76,0.705914,0.990563,0.972352,101


In [8]:
# combine medsam yolo results
df_medsam_yolo = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_medsam_with_yolo_main_10-14-24/val/*/0.001/*.pkl', get_label_num = lambda x: int(x.split('/')[-3]))
df_medsam_yolo = df_medsam_yolo.reset_index(drop=True)
df_medsam_yolo.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_medsam_with_yolo_main_10-14-24/val/combined_val_INCOMPLETE.pkl')

In [8]:
df_combined_sam = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_SAM_training_all_labels_10-18-24/val/*/0.001/eval_singletask_unprompted_val_label*.pkl', get_label_num = lambda x: int(x.split('/')[-3]))
df_combined_sam = df_combined_sam.reset_index(drop=True)
df_combined_sam.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_SAM_training_all_labels_10-18-24/val/combined_val.pkl')

In [3]:
df_combined_medsam_yolo_untrained = combine_into_one_df('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_untrained_medsam_yolo_10-22-24/val/*/0.001/eval_singletask_yolov7_prompted_val_label*.pkl', get_label_num = lambda x: int(x.split('/')[-3]))
df_combined_medsam_yolo_untrained = df_combined_medsam_yolo_untrained.reset_index(drop=True)
df_combined_medsam_yolo_untrained.to_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/singletask_untrained_medsam_yolo_10-22-24/val/combined_val.pkl')